# Lista 1

In [3]:
import pandas as pd
import quadprog

## Questão 1

Vamos resolver o modelo de Markowitz com a função utilidade quadrática (que na verdade é derivada da função utilidade exponencial). Considere o modelo dado no slide 14 de Black-Litterman, com $λ= 3.07$. Note que ao implementar este modelo em algum pacote (tipo o quadprog) não é necessário dividir por 2 pois normalmente isto está embutido nos solvers. Calcule $Σ$ através dos dados históricos.

In [14]:
# carrega dados
data = pd.read_csv("data/IBOV.csv")
data["Date"] = pd.to_datetime(data["Date"], format="%Y%m%d")
data = data.set_index("Date")

# remove IBOV
if "IBOV" in data:
    del data["IBOV"]

# calcula retornos
data = data.pct_change().dropna()
    
data.head()

,BOVA11,ABEV3,AZUL4,BTOW3,B3SA3,BBAS3,BBSE3,BRML3,BBDC3,BBDC4,...,SANB11,CSNA3,SULA11,SUZB3,TAEE11,TIMP3,TOTS3,USIM5,VALE3,WEGE3
Date,,,,,,,,,,,,,,,,,,,,,
2018-04-25,-0.004850,-0.006343,-0.044909,-0.008570,-0.009791,0.013293,0.000452,0.010956,0.015490,0.011864,...,-0.030586,0.001175,0.022124,0.039881,0.008413,-0.010183,-0.007656,0.012464,-0.007433,-0.021327
2018-04-26,0.017422,0.006384,0.003213,-0.008976,0.024506,0.007752,0.012664,0.019704,0.001795,0.000000,...,-0.001461,0.009390,0.011063,0.020472,-0.011919,0.028121,0.009643,0.016098,0.015639,0.047821
2018-04-27,-0.000479,-0.005890,0.030859,-0.012747,0.006295,0.000000,0.005360,-0.011594,0.002687,0.011307,...,0.005266,-0.034884,-0.004757,0.010919,0.018697,0.002668,0.001910,-0.027027,-0.007157,0.000578
2018-04-30,-0.003474,0.006381,0.027958,-0.032620,-0.020434,-0.017456,-0.019103,-0.009775,-0.019652,-0.015735,...,-0.002910,-0.013253,-0.015296,0.020095,-0.000592,0.000000,-0.001907,0.017241,0.006772,-0.008083
2018-05-02,-0.017673,-0.028080,-0.024451,-0.038988,-0.027246,-0.018368,0.000000,-0.024679,-0.015945,-0.025663,...,-0.031524,0.014652,-0.005825,0.033982,-0.002962,-0.029940,-0.026743,-0.006591,0.000651,0.012806


In [12]:
# calcula retornos esperados
expected_returns = data.mean()
display(expected_returns.head())

# calcula matriz de covariância
cov_matrix = data.cov()
display(cov_matrix.head())

BOVA11    54.563397
ABEV3     11.235315
AZUL4      8.122847
BTOW3     29.408447
B3SA3     13.462775
dtype: float64

,BOVA11,ABEV3,AZUL4,BTOW3,B3SA3,BBAS3,BBSE3,BRML3,BBDC3,BBDC4,...,SANB11,CSNA3,SULA11,SUZB3,TAEE11,TIMP3,TOTS3,USIM5,VALE3,WEGE3
BOVA11,721.526831,100.177757,272.405942,-70.094598,257.409361,218.465920,177.691488,66.733377,126.300545,143.430404,...,280.485126,4.008827,246.512308,250.123271,148.794130,43.155241,94.888446,-83.255143,149.521182,153.015173
ABEV3,100.177757,36.787298,53.526735,-33.393192,46.427162,43.451037,60.866671,15.078566,32.974726,32.760832,...,58.100307,-15.467802,44.800728,44.162750,38.065763,12.091599,19.286994,-34.181890,7.795660,33.355576
AZUL4,272.405942,53.526735,238.588800,93.234297,157.523191,153.543861,118.301938,26.608980,90.417363,101.124235,...,175.815409,-0.062691,158.736547,196.098684,95.370526,31.786511,43.064065,-26.524255,115.198890,103.079182
BTOW3,-70.094598,-33.393192,93.234297,428.728588,80.448849,56.402170,15.528891,-19.044152,20.146830,26.744022,...,28.720733,30.679337,83.587625,127.852025,25.341936,16.093231,11.083146,63.307067,115.878321,89.956463
B3SA3,257.409361,46.427162,157.523191,80.448849,147.840113,115.262898,105.352539,24.796098,69.897428,76.970500,...,138.235868,-3.288194,142.259506,150.328351,86.266638,25.824946,47.902489,-37.516017,82.469464,105.333484


**(a)** Encontre o portfolio P1 que otimiza o modelo acima, calculando $μ$ a partir dos retornos históricos. Calcule $μ_{P1}$ e $σ_{P1}$.

**(b)** Encontre o portfolio P2 utilizando $μ$ como o vetor de retornos implícitos necessários para obter oportfolio de mercado no problema irrestrito (slides 18 e 19). Para os pesos do mercado, utilize o arquivo fornecido aqui. Calcule $μ_{P2}$ e $σ_{P2}$.

**(c)** Suponha que você possuia uma bola de cristal no final de 2019, e sabia que, até Agosto de 2020, os retornos médios diários dos ativos WEGE3 e MRFG3 seriam 0.0055754 e 0.0037247. Utilize o modelode Black-Litterman com estas previsões para encontrar um novo vetor de retornos esperados $μ$, e encontre o portfolio P3. Calcule $μ_{P3}$ e $σ_{P3}$. Considere $τ= 0.025$.

**(d)** Calcule as séries out-of-sample dos três portfolios assumindo um investimento inicial de R\$1. Plote um gráfico comparativo com a performance dos mesmos no período out-of-sample. Inclua também a série do índice iBov --  normalize o iBov para comecar de 1. Calcule  também o retorno médio diário, o desvio padrão diário e o valor final das séries out-of-sample para cada portfolio. 

## Questão 2 

Nesta questão vamos comparar a performance de dois modelos: Markowitz (minimizando variância) e CVaR (Slide 58 de Downside Risk) com $α= 5\%$. Em ambos os modelos, utilize 0.01% (0.0001) como retorno mínimo diário e 15% como peso máximo que um ativo qualquer pode ter no portfolio. Para o Markowitz, calcule $μ$ e $Σ$ através dos dados históricos. Para o CVaR, utilize a matriz de retornos in-sample como cenários (pela minha resolução foram 249 cenários). Para as séries out-of-sample dos dois portfolios:

**(a)** Plote um gráfico comparativo com a performance dos portfolios (simulando investimento inicial de R\$1) e do índice iBov normalizado para começar de 1.

**(b)** Complete a tabela abaixo: 

| | | | | | |
| :- | :- | :- | :- | :- | :- | 
| Portfolio | Retorno esperado | Desvio padrão | CVaR 5\% | Sharpe ratio | STARR ratio 5\% | Drawdown máximo | 
| IBOV | | | | | |
| Markowitz | | | | | |
| CVaR | | | | | |

## Questão  3

Nesta  questão  incluiremos  aspectos  práticos  nos  portfolios, fazendo uma simulação mais realista. Escolha o modelo de seleção de portfolios que quiser. Faça rebalanceamento a cada 5 dias com recálculo de estratégia, ou seja, resolvendo o modelo novamente para ter uma nova solução. Utilizecomo período in-sample uma janela deslizante do mesmo tamanho dos dadosin-sample originais. Inclua também as seguintes restrições no modelo de otimização: 
- Peso máximo de 10% por ativo.
- Se o ativo for escolhido, peso mínimo de 0.5%.
- Turnover máximo de 30% por rebalanceamento.
- Após escolher os pesos ótimos, simule as compras e vendas a partir do modelo que minimiza custos de transação (Slide 32 de Simulação). Você pode escolher como investimento inicial o valor que quiser (ex.:  R$500 mil).
- Se estiver animado, restrinja as compras e vendas para serem sempre em lotes de 100 (substituindo o modelo do Slide 32 pelo do Slide 61). Plote o gráfico de performance do seu portfolio, e calcule algumas métricas de performance. Você pode, se quiser, experimentar com diferentes parâmetros para as restrições acima e ver como afetam a performancedo portfolio. 

## Desafio

Considere o modelo de otimização abaixo. O modelo assume, para cada ativo i, um portfolio atual composto por $X_i$ ações e proporções desejadas $w^{∗}_{i}$, encontradas previamente, indicando como devemos dividir o novo portfolio. Considere também $P_i$ como o preço atual de i e f como o custo aplicado a cada transação e expresso como uma porcentagem do valor negociado. O modelo utiliza variáveis $x_i$ indicando quantas ações de i teremos após as negociações e $G_i$ indicandoo valor financeiro gasto para alterarmos a composição deide $X_i$ para $x_i$ ações. Este modelo garante que gastaremos o mínimo possível nestes custos. 

$min \sum_{i=1}^{N} G_i$ 

$sujeito\quad a\quad P_i x_i = w∗i(\sum_{i=1}^{N} X_i P_i − \sum_{j=1}^{N} G_j)\quad i=1,...,N $

$\quad \quad \quad \quad \quad G_i ≥ (x_i − X_i) P_if\quad i= 1,...,N$

$\quad \quad \quad \quad \quad G_i ≥ (X_i − x_i) P_if i= 1,...,N$

O modelo inclui apenas custos variáveis (uma porcentagem do valor a ser negociado). Considere que, além do custo variável, temos um custo fixo h, expresso em moeda, a ser aplicado a cada negociação. Este valor é independente do tamanho da negociação. Altere o modelo acima para que inclua o custo fixo.